In [1]:
import itertools
from itertools import permutations

import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

%matplotlib inline

## Input data

* Tox21 compounds with cluster assignment 
* Compound to target annotations from pharos and drug hub
* Target enrichment results for each cluster

In [9]:
tox21_data = pd.read_csv("../data/data_w_identifiers.csv").iloc[: , 1:]
enrichment_results = pd.read_csv("../data/enrichment_analysis_results.csv").iloc[: , 1:]

In [10]:
tox21_data.head()

,cluster,cas,pubchem_cid,sample_name,smiles,parent_smiles,inchikey,target_gene,approved_name
0,1,129299-90-7,180494.0,Fabesetron hydrochloride,Cl.Cc4ncnc4C[C@H]3CCc2c(C)c1ccccc1n2C3=O,CC1=C(CC2CCC3=C(C)C4=CC=CC=C4N3C2=O)NC=N1,AEKQMJRJRAHOAP-UHFFFAOYSA-N,NaN,"symbol withdrawn, see [HGNC:12811](/data/gene-..."
1,1,NOCAS_48522,60196348.0,HMR1171 trifluoroacetate (1:1),FC(F)(F)C(=O)O.CCN(CC)C(=O)c1cc(c(cc1N(CC)CCN(...,CCN(CC)C(=O)C1=CC(=C(C=C1N(CC)CCN(C)C)N1CCC(CC...,AUEUBSJNKBZSKK-UHFFFAOYSA-N,NaN,"symbol withdrawn, see [HGNC:12811](/data/gene-..."
2,1,33414-30-1,969494.0,Ftormetazine,CN1CCN(CC1)CCC(=O)N2c4ccccc4Sc3ccc(cc23)C(F)(F)F,CN1CCN(CCC(=O)N2C3=CC=CC=C3SC3=CC=C(C=C23)C(F)...,DOUQJBPSTIKRPH-UHFFFAOYSA-N,NaN,"symbol withdrawn, see [HGNC:12811](/data/gene-..."
3,1,289716-94-5,9861124.0,CP-607366,Clc2ccc(Oc1ccc(F)cc1CNC)cc2Cl,CNCC1=C(OC2=CC=C(Cl)C(Cl)=C2)C=CC(F)=C1,FQEBOQLYHASAOY-UHFFFAOYSA-N,SLC6A4,solute carrier family 6 member 4
4,1,289716-94-5,9861124.0,CP-607366,Clc2ccc(Oc1ccc(F)cc1CNC)cc2Cl,CNCC1=C(OC2=CC=C(Cl)C(Cl)=C2)C=CC(F)=C1,FQEBOQLYHASAOY-UHFFFAOYSA-N,KCNH2,potassium voltage-gated channel subfamily H me...


In [11]:
enrichment_results.head()

,cluster,target_gene,approved_name,TruePos,FalsePos,FalseNeg,TrueNeg,TotalCmpdsTested,OddsRatio,p_val,significant
0,1,ABCB1,ATP binding cassette subfamily B member 1,1,24,26,1778,1829,2.849359,0.312219,False
1,1,ACHE,acetylcholinesterase (Cartwright blood group),1,62,26,1740,1829,1.079404,0.614543,False
2,1,ADRA1A,adrenoceptor alpha 1A,4,79,23,1723,1829,3.793065,0.031174,False
3,1,ADRA1B,adrenoceptor alpha 1B,4,56,23,1746,1829,5.422360,0.010440,False
4,1,ADRA1D,adrenoceptor alpha 1D,8,64,19,1738,1829,11.434211,0.000005,True


## Filter and merge

* Filter enrichments based on p-value (TODO: use better cutoff)
* Merge target and cluster data

In [12]:
tox21_data = tox21_data[['cluster','cas','pubchem_cid','sample_name','inchikey','target_gene']]
target_data

In [13]:
tox21_data

,cluster,cas,pubchem_cid,sample_name,inchikey,target_gene
0,1,129299-90-7,180494.0,Fabesetron hydrochloride,AEKQMJRJRAHOAP-UHFFFAOYSA-N,NaN
1,1,NOCAS_48522,60196348.0,HMR1171 trifluoroacetate (1:1),AUEUBSJNKBZSKK-UHFFFAOYSA-N,NaN
2,1,33414-30-1,969494.0,Ftormetazine,DOUQJBPSTIKRPH-UHFFFAOYSA-N,NaN
3,1,289716-94-5,9861124.0,CP-607366,FQEBOQLYHASAOY-UHFFFAOYSA-N,SLC6A4
4,1,289716-94-5,9861124.0,CP-607366,FQEBOQLYHASAOY-UHFFFAOYSA-N,KCNH2
...,...,...,...,...,...,...
14826,142,10325-94-7,11402127.0,Cadmium dinitrate,XIEPJMXMMWZAAV-UHFFFAOYSA-N,NaN
14827,142,59-85-8,1730.0,4-Chloromercuribenzoic acid,YFZOUMNUDGGHIW-UHFFFAOYSA-M,NaN
14828,143,NOCAS_45111,443939.0,Epirubicin hydrochloride,AOJJSUZBOXZQNB-TZSSRYMLSA-N,TOP2A
14829,143,NOCAS_45111,443939.0,Epirubicin hydrochloride,AOJJSUZBOXZQNB-TZSSRYMLSA-N,S100A4


In [25]:
cutoff=0.008570463#0.05/enrichment_results.shape[0]
significant_enrichments = enrichment_results[(enrichment_results['p_val']<cutoff) & (enrichment_results['TruePos']>1)]


In [26]:
significant_enrichments

,cluster,target_gene,approved_name,TruePos,FalsePos,FalseNeg,TrueNeg,TotalCmpdsTested,OddsRatio,p_val,significant
4,1,ADRA1D,adrenoceptor alpha 1D,8,64,19,1738,1829,11.434211,0.000005,True
5,1,ADRA2A,adrenoceptor alpha 2A,6,76,21,1726,1829,6.488722,0.000939,True
6,1,ADRA2B,adrenoceptor alpha 2B,7,72,20,1730,1829,8.409722,0.000095,True
7,1,ADRA2C,adrenoceptor alpha 2C,5,71,22,1731,1829,5.540973,0.004274,True
15,1,CALM1,calmodulin 1,4,8,23,1794,1829,39.000000,0.000017,True
...,...,...,...,...,...,...,...,...,...,...,...
6400,144,CA2,carbonic anhydrase 2,3,63,9,1754,1829,9.280423,0.007820,True
6401,144,CA9,carbonic anhydrase 9,3,49,9,1768,1829,12.027211,0.003980,True
6405,144,HSP90AA1,heat shock protein 90 alpha family class A mem...,2,11,10,1806,1829,32.836364,0.002958,True
6409,144,METAP2,methionyl aminopeptidase 2,2,5,10,1812,1829,72.480000,0.000814,True


# Find novel compound - target candidates 

* A novel compound-target pair candidate is defined as:
    * The compound is in the cluster
    * The target is in enriched in the cluster
    * The compound is not known have activity against the target
* I will then tier the candidate pairs in the following way:
    * Tier 1: If the compound is known to have activity against a different enriched target
        * Justification: the enriched targets may be related and so activity against one may suggest activity against another 
    * Tier 2: If the compound is known to have activity against ANY target
        * Justification: if a compound has a target annotation then it has been studied in a bioassay and at least has some biological activity, thus is more likely to have activity against other targets too
    * Tier 3: Pairs with compounds having no annotation
* Within each tier, the candidate pairs could be sorted by the number of compounds with known activity against the target -- but this makes sense as a global analysis not within tox21 compounds only.

In [35]:
compound_targets = {}
for i, row in tox21_data.dropna(subset=['target_gene']).iterrows():
    if (row['cas'],row['inchikey']) in compound_targets:
        compound_targets[(row['cas'],row['inchikey'])].append(row['target_gene'])
    else:
        compound_targets[(row['cas'],row['inchikey'])] = [row['target_gene']]


In [37]:
#not every cluster has significant targets
candidates = []
for focal_cluster in set(significant_enrichments['cluster']):
    focal_cluster_compounds = tox21_data[tox21_data['cluster']==focal_cluster]
    focal_cluster_enrichments = significant_enrichments[significant_enrichments['cluster']==focal_cluster]
        
    for c, compound_row in focal_cluster_compounds.iterrows():
        for t, target_row in focal_cluster_enrichments.iterrows():
            #confirm this compound - target pair is not an existing annotations
            candidate_row = {**dict(compound_row),**dict(target_row)}
            compound_annot = compound_targets.get((compound_row['cas'],compound_row['inchikey']))
            if compound_annot is not None:
                
                if target_row['target_gene'] in compound_annot:
                    #compound has known activity against this target
                    pass
                elif len(set(focal_cluster_enrichments['target_gene']).intersection(set(compound_annot)))>0:
                    #compound has activity against a different enriched target 
                    candidate_row['tier'] = 'Tier 1'
                    candidates.append(candidate_row)
                else:
                    #compound has activity against another target (not enriched in this cluster)
                    candidate_row['tier'] = 'Tier 2'
                    candidates.append(candidate_row)
                
            else:
                candidate_row['tier'] = 'Tier 3'
                candidates.append(candidate_row)      
                
    
    
compound_target_candidates = pd.DataFrame(candidates)


In [43]:
compound_target_candidates = compound_target_candidates.drop_duplicates()

In [44]:
compound_target_candidates[compound_target_candidates['tier']=='Tier 1']

,cluster,cas,pubchem_cid,sample_name,inchikey,target_gene,approved_name,TruePos,FalsePos,FalseNeg,TrueNeg,TotalCmpdsTested,OddsRatio,p_val,significant,tier
81,1,289716-94-5,9861124.0,CP-607366,FQEBOQLYHASAOY-UHFFFAOYSA-N,ADRA1D,adrenoceptor alpha 1D,8,64,19,1738,1829,11.434211,0.000005,True,Tier 1
82,1,289716-94-5,9861124.0,CP-607366,FQEBOQLYHASAOY-UHFFFAOYSA-N,ADRA2A,adrenoceptor alpha 2A,6,76,21,1726,1829,6.488722,0.000939,True,Tier 1
83,1,289716-94-5,9861124.0,CP-607366,FQEBOQLYHASAOY-UHFFFAOYSA-N,ADRA2B,adrenoceptor alpha 2B,7,72,20,1730,1829,8.409722,0.000095,True,Tier 1
84,1,289716-94-5,9861124.0,CP-607366,FQEBOQLYHASAOY-UHFFFAOYSA-N,ADRA2C,adrenoceptor alpha 2C,5,71,22,1731,1829,5.540973,0.004274,True,Tier 1
85,1,289716-94-5,9861124.0,CP-607366,FQEBOQLYHASAOY-UHFFFAOYSA-N,CALM1,calmodulin 1,4,8,23,1794,1829,39.000000,0.000017,True,Tier 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84699,144,140-89-6,2735045.0,Potassium ethyl xanthate,ZOOODBUHSVUZEM-UHFFFAOYSA-N,METAP2,methionyl aminopeptidase 2,2,5,10,1812,1829,72.480000,0.000814,True,Tier 1
84700,144,140-89-6,2735045.0,Potassium ethyl xanthate,ZOOODBUHSVUZEM-UHFFFAOYSA-N,PTGS2,prostaglandin-endoperoxide synthase 2,3,64,9,1753,1829,9.130208,0.008156,True,Tier 1
84710,144,140-90-9,23690437.0,Sodium O-ethyl carbonodithioate,ZOOODBUHSVUZEM-UHFFFAOYSA-N,HSP90AA1,heat shock protein 90 alpha family class A mem...,2,11,10,1806,1829,32.836364,0.002958,True,Tier 1
84711,144,140-90-9,23690437.0,Sodium O-ethyl carbonodithioate,ZOOODBUHSVUZEM-UHFFFAOYSA-N,METAP2,methionyl aminopeptidase 2,2,5,10,1812,1829,72.480000,0.000814,True,Tier 1


In [40]:
#Confirm no overlap between candidate set and target annotations
compound_target_candidates[['cas','inchikey','target_gene']].merge(tox21_data[['cas','inchikey','target_gene']],how='inner')

,cas,inchikey,target_gene


In [45]:
compound_target_candidates.to_csv("../data/tox21_cluster_compound_target_candidates.csv",index=False)